## FINETUNING da la derniere couche

In [6]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm

# Fonction de collate personnalisée pour garder les images sous forme de liste PIL
def custom_collate_fn(batch):
    images, labels = zip(*batch)
    return list(images), torch.tensor(labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Préparer le dataset CIFAR-10 avec transformation (redimensionnement à 224x224)
transform = transforms.Resize((224, 224))
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Réduire le train set à 30% de ses données
total_train = len(train_dataset)
train_size = int(0.3 * total_train)
train_subset, _ = random_split(train_dataset, [train_size, total_train - train_size])

# Créer les DataLoaders en utilisant la fonction de collate personnalisée
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

# Charger le processor et le modèle ViT pré-entraîné
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
# Remplacer la couche finale pour qu'elle produise 10 classes au lieu de 1000
model.classifier = nn.Linear(model.config.hidden_size, 10)
model.to(device)

# Définir la loss et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Boucle d'entraînement
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        # Le processor attend une liste d'images PIL
        inputs = processor(images=images, return_tensors="pt").to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss moyenne: {running_loss/len(train_loader):.4f}")

# Évaluation sur le jeu de test
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluation"):
        inputs = processor(images=images, return_tensors="pt").to(device)
        outputs = model(**inputs)
        preds = outputs.logits.argmax(dim=1)
        correct += (preds == labels.to(device)).sum().item()
        total += labels.size(0)

print("Accuracy sur CIFAR-10: {:.2f}%".format(100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified


Epoch 1/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [02:18<00:00,  3.39it/s]


Epoch 1/3 - Loss moyenne: 0.1764


Epoch 2/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [02:18<00:00,  3.38it/s]


Epoch 2/3 - Loss moyenne: 0.0602


Epoch 3/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [02:18<00:00,  3.39it/s]


Epoch 3/3 - Loss moyenne: 0.0328


Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:41<00:00,  7.59it/s]

Accuracy sur CIFAR-10: 97.27%


## FINETUNING des 3 dernieres couches

In [7]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm

# Fonction de collate personnalisée pour garder les images sous forme de liste PIL
def custom_collate_fn(batch):
    images, labels = zip(*batch)
    return list(images), torch.tensor(labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Préparer le dataset CIFAR-10 avec transformation (redimensionnement à 224x224)
transform = transforms.Resize((224, 224))
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Réduire le train set à 30% de ses données pour un test rapide
total_train = len(train_dataset)
train_size = int(0.1 * total_train)
train_subset, _ = random_split(train_dataset, [train_size, total_train - train_size])

# Créer les DataLoaders en utilisant la fonction de collate personnalisée
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

# Charger le processor et le modèle ViT pré-entraîné
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

# ----------------- Fine-Tuning sur les dernières couches -----------------
# 1. Geler tout d'abord le backbone
for param in model.vit.parameters():
    param.requires_grad = False

# 2. Débloquer (unfreeze) les derniers blocs du transformeur.
# Ici, on choisit de réentraîner les 2 derniers blocs
for layer in model.vit.encoder.layer[-2:]:
    for param in layer.parameters():
        param.requires_grad = True

# 3. Remplacer la tête de classification pour 10 classes (pour CIFAR-10)
model.classifier = nn.Linear(model.config.hidden_size, 10)
# Par défaut, les paramètres de cette nouvelle couche sont entraînables

model.to(device)

# Définir la loss et l'optimiseur.
# L'optimiseur portera sur tous les paramètres non gelés (les 2 derniers blocs et le classifier)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Boucle d'entraînement
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        # Préparation des inputs via le processor (attend une liste d'images PIL)
        inputs = processor(images=images, return_tensors="pt").to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss moyenne: {running_loss/len(train_loader):.4f}")

# Évaluation sur le jeu de test
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluation"):
        inputs = processor(images=images, return_tensors="pt").to(device)
        outputs = model(**inputs)
        preds = outputs.logits.argmax(dim=1)
        correct += (preds == labels.to(device)).sum().item()
        total += labels.size(0)

print("Accuracy sur CIFAR-10: {:.2f}%".format(100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified


Epoch 1/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.42it/s]


Epoch 1/3 - Loss moyenne: 0.4037


Epoch 2/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.36it/s]


Epoch 2/3 - Loss moyenne: 0.0626


Epoch 3/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.33it/s]


Epoch 3/3 - Loss moyenne: 0.0185


Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:40<00:00,  7.66it/s]

Accuracy sur CIFAR-10: 97.24%


In [15]:
size_baseline = get_model_size(model)
print("Taille du modèle baseline: {:.2f} MB".format(size_baseline))

Taille du modèle baseline: 343.29 MB


## QUANTIZATION

In [14]:
import copy

# Copier le modèle fine-tuné
model_quantized = copy.deepcopy(model)

# Quantization dynamique du classifier
model_quantized.classifier = torch.quantization.quantize_dynamic(
    model_quantized.classifier, {nn.Linear}, dtype=torch.qint8
)
model_quantized.to("cpu")

# Évaluation du modèle quantized
correct_q, total_q = 0, 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluation Quantized"):
        inputs = processor(images=images, return_tensors="pt")
        outputs = model_quantized(**inputs)
        preds = outputs.logits.argmax(dim=1)
        correct_q += (preds == labels).sum().item()
        total_q += labels.size(0)

accuracy_quantized = 100 * correct_q / total_q
size_quantized = get_model_size(model_quantized)

print("\n⚡ Résultats - QUANTIZATION (tête uniquement)")
print(f"Accuracy : {accuracy_quantized:.2f}%")
print(f"Taille du modèle : {size_quantized:.2f} MB")

Evaluation Quantized:   0%|                                                                                                                                                    | 0/313 [00:00<?, ?it/s]

Evaluation Quantized: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [03:32<00:00,  1.47it/s]



⚡ Résultats - QUANTIZATION (tête uniquement)
Accuracy : 97.24%
Taille du modèle : 343.29 MB


## PRUNING1

In [9]:
import torch.nn.utils.prune as prune

# Créer une copie du modèle fine-tuné pour le pruning
model_pruned = copy.deepcopy(model)

# Appliquer du pruning sur la tête de classification
prune.random_unstructured(model_pruned.classifier, name="weight", amount=0.3)

# Optionnel : Appliquer du pruning sur les Linear des 2 derniers blocs du transformeur
for layer in model_pruned.vit.encoder.layer[-2:]:
    for module in layer.modules():
        if isinstance(module, nn.Linear):
            prune.random_unstructured(module, name="weight", amount=0.3)

# Évaluation du modèle pruned sur le jeu de test
model_pruned.eval()
correct_p = 0
total_p = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluation Pruned"):
        inputs = processor(images=images, return_tensors="pt").to(device)
        outputs = model_pruned(**inputs)
        preds = outputs.logits.argmax(dim=1)
        correct_p += (preds == labels.to(device)).sum().item()
        total_p += labels.size(0)
print("Accuracy du modèle pruned: {:.2f}%".format(100 * correct_p / total_p))

# Comparaison des tailles
size_pruned = get_model_size(model_pruned)
print("Taille du modèle pruned: {:.2f} MB".format(size_pruned))

Evaluation Pruned: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:44<00:00,  6.98it/s]


Accuracy du modèle pruned: 93.39%
Taille du modèle pruned: 399.95 MB


## PRUNING

In [18]:
import copy
import torch.nn.utils.prune as prune

# Copier le modèle fine-tuné
model_pruned = copy.deepcopy(model)

# Appliquer du pruning non structuré sur la tête de classification
prune.random_unstructured(module, name="weight", amount=0.3)
prune.remove(module, 'weight')  # Supprime le mask et applique le pruning définitivement

# Optionnel : pruning sur les 2 derniers blocs ViT
for layer in model_pruned.vit.encoder.layer[-2:]:
    for module in layer.modules():
        if isinstance(module, nn.Linear):
            prune.random_unstructured(module, name="weight", amount=0.3)

model_pruned.to("cpu")

# Évaluation du modèle pruned
correct_p, total_p = 0, 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluation Pruned"):
        inputs = processor(images=images, return_tensors="pt")
        outputs = model_pruned(**inputs)
        preds = outputs.logits.argmax(dim=1)
        correct_p += (preds == labels).sum().item()
        total_p += labels.size(0)

accuracy_pruned = 100 * correct_p / total_p
size_pruned = get_model_size(model_pruned)

print("\n📉 Résultats - PRUNING")
print(f"Accuracy : {accuracy_pruned:.2f}%")
print(f"Taille du modèle : {size_pruned:.2f} MB")


Evaluation Pruned:   0%|                                                                                                                                                       | 0/313 [00:00<?, ?it/s]

Evaluation Pruned: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [03:10<00:00,  1.64it/s]



📉 Résultats - PRUNING
Accuracy : 94.96%
Taille du modèle : 399.92 MB


## COMPARAISON

In [21]:
print("\n📊 RÉCAPITULATIF DES MODÈLES\n")
print("┌──────────────────────────────┬────────────┬────────────┐")
print("│ Modèle                       │ Accuracy   │ Taille (MB)│")
print("├──────────────────────────────┼────────────┼────────────┤")
print(f"│ Baseline (fine-tuné)         │ {100 * correct / total:8.2f}% │ {size_baseline:10.2f} │")
print(f"│ Quantized (tête uniquement)  │ {accuracy_quantized:8.2f}% │ {size_quantized:10.2f} │")
print(f"│ Pruned (tête + 2 derniers)   │ {accuracy_pruned:8.2f}% │ {size_pruned:10.2f} │")
print("└──────────────────────────────┴────────────┴────────────┘")



📊 RÉCAPITULATIF DES MODÈLES

┌──────────────────────────────┬────────────┬────────────┐
│ Modèle                       │ Accuracy   │ Taille (MB)│
├──────────────────────────────┼────────────┼────────────┤
│ Baseline (fine-tuné)         │    97.24% │     343.29 │
│ Quantized (tête uniquement)  │    97.24% │     343.29 │
│ Pruned (tête + 2 derniers)   │    94.96% │     399.92 │
└──────────────────────────────┴────────────┴────────────┘
